In [ ]:
#%%timeit
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import process, fuzz

# --- entity resolution --- #
resolution_threshold = 84
unique_remarks = remark_df['Remarks'].unique().tolist()
score_sort = [(x,) + i
             for x in unique_remarks 
             for i in process.extract(x, unique_remarks, scorer=fuzz.token_sort_ratio)]
similarity_sort = pd.DataFrame(score_sort, columns=['remarks_sort','match_sort','score_sort'])
similarity_sort['sorted_remarks_sort'] = np.minimum(similarity_sort['remarks_sort'], similarity_sort['match_sort'])
#Pick values
high_score_set = similarity_sort[(similarity_sort['score_sort'] >= resolution_threshold) & 
                                    (similarity_sort['remarks_sort'] != similarity_sort['match_sort']) & 
                                    (similarity_sort['sorted_remarks_sort'] != similarity_sort['match_sort'])]
#Drop the representative value column
high_score_set = high_score_set.drop('sorted_remarks_sort',axis=1).copy()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
final_reso_set = high_score_set.groupby(['match_sort','score_sort']).agg(
                       {'remarks_sort': ', '.join}).sort_values(
                       ['score_sort'], ascending=False)